In [69]:
import re
import pandas as pd
import numpy as np
from collections import Counter 
from konlpy.tag import Okt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

In [70]:
# 기존 X_train csv 파일 전환 후 로드시 X_train.csv 파일로 틀어짐 전처리 
X_train = pd.read_csv('X_train.csv')
X_train = X_train.drop('Unnamed: 0' , axis=1)
X_train2 = X_train["tokenized"].squeeze()
X_train3 = X_train2.values
X_train4 = []
for str in X_train3:
    str = str.replace("[", "").replace("]", "").replace(" '", "").replace("'", "")
    X_train4.append(str.split(","))
X_train = X_train4.copy()

In [71]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [72]:
## 재 토큰화  # vocab_szie 1756
tokenizer = Tokenizer(1756, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)




In [73]:
# 리뷰길이 25으로 패딩 , max_len = 25 확인 
X_train = pad_sequences(X_train, maxlen=25)

In [74]:
# 모델로드
loaded_model = load_model('test01_model.h5')

In [ ]:
## 리뷰예측하기
# 형태소 처리 라이브러리 정의 
okt =  Okt()
# 불용어 : 추가시 리스트에 추가
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

def sentiment_predict(new_sentence):
  new_sentence = new_sentence.replace("↑", "상승").replace("↓", "하락")
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = 25)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [75]:
sentiment_predict('제2의 리먼사태 루나코인사태에 증시 한때 하락')

99.66% 확률로 부정 리뷰입니다.


In [79]:
sentiment_predict('유럽증시 전쟁연기 소식 15% ↑')

99.02% 확률로 긍정 리뷰입니다.


In [80]:
sentiment_predict('2호 회사 영업실적 최대 기대감 솔솔')

100.00% 확률로 긍정 리뷰입니다.


In [81]:
sentiment_predict('2호 회사 신규 상품 출시')

96.91% 확률로 긍정 리뷰입니다.


In [82]:
sentiment_predict('코스피 공매도 및 외국인 매도에 하락 진행 중 ')

99.99% 확률로 부정 리뷰입니다.


In [83]:
sentiment_predict(' 오를까 내릴까 결국 내렸지 하지만 오르길 기대하는 뉴스')

63.35% 확률로 긍정 리뷰입니다.
